In [37]:
from rectpack.maxrects import MaxRectsBssf

import operator
import itertools
import collections

import decimal

In [38]:
class BinFactory(object):

    def __init__(self, width, height, count, pack_algo, *args, **kwargs):
        self._width = width
        self._height = height
        self._count = count
        
        self._pack_algo = pack_algo
        self._algo_kwargs = kwargs
        self._algo_args = args
        self._ref_bin = None # Reference bin used to calculate fitness
        
        self._bid = kwargs.get("bid", None)

    def _create_bin(self):
        return self._pack_algo(self._width, self._height, *self._algo_args, **self._algo_kwargs)

    def is_empty(self):
        return self._count<1

    def fitness(self, width, height):
        if not self._ref_bin:
            self._ref_bin = self._create_bin()

        return self._ref_bin.fitness(width, height)

    def fits_inside(self, width, height):
        # Determine if rectangle widthxheight will fit into empty bin
        if not self._ref_bin:
            self._ref_bin = self._create_bin()

        return self._ref_bin._fits_surface(width, height)

    def new_bin(self):
        if self._count > 0:
            self._count -= 1
            return self._create_bin()
        else:
            return None

    def __eq__(self, other):
        return self._width*self._height == other._width*other._height

    def __lt__(self, other):
        return self._width*self._height < other._width*other._height

    def __str__(self):
        return "Bin: {} {} {}".format(self._width, self._height, self._count)

In [39]:
class PackerOnline(object):
    """
    Rectangles are packed as soon are they are added
    """

    def __init__(self, pack_algo=MaxRectsBssf, rotation=False):
        """
        Arguments:
            pack_algo (PackingAlgorithm): What packing algo to use
            rotation (bool): Enable/Disable rectangle rotation
        """
        self._rotation = rotation
        self._pack_algo = pack_algo
        self.reset()

    def __iter__(self):
        return itertools.chain(self._closed_bins, self._open_bins)

    def __len__(self):
        return len(self._closed_bins)+len(self._open_bins)
    
    def __getitem__(self, key):
        """
        Return bin in selected position. (excluding empty bins)
        """
        if not isinstance(key, int):
            raise TypeError("Indices must be integers")

        size = len(self)  # avoid recalulations

        if key < 0:
            key += size

        if not 0 <= key < size:
            raise IndexError("Index out of range")
        
        if key < len(self._closed_bins):
            return self._closed_bins[key]
        else:
            return self._open_bins[key-len(self._closed_bins)]

    def _new_open_bin(self, width=None, height=None, rid=None):
        """
        Extract the next empty bin and append it to open bins
        Returns:
            PackingAlgorithm: Initialized empty packing bin.
            None: No bin big enough for the rectangle was found
        """
        factories_to_delete = set() #
        new_bin = None

        for key, binfac in self._empty_bins.items():

            # Only return the new bin if the rect fits.
            # (If width or height is None, caller doesn't know the size.)
            if not binfac.fits_inside(width, height):
                continue
           
            # Create bin and add to open_bins
            new_bin = binfac.new_bin()
            if new_bin is None:
                continue
            self._open_bins.append(new_bin)

            # If the factory was depleted mark for deletion
            if binfac.is_empty():
                factories_to_delete.add(key)
       
            break

        # Delete marked factories
        for f in factories_to_delete:
            del self._empty_bins[f]

        return new_bin 

    def add_bin(self, width, height, count=1, **kwargs):
        # accept the same parameters as PackingAlgorithm objects
        kwargs['rot'] = self._rotation
        bin_factory = BinFactory(width, height, count, self._pack_algo, **kwargs)
        self._empty_bins[next(self._bin_count)] = bin_factory

    def rect_list(self):
        rectangles = []
        bin_count = 0

        for abin in self:
            for rect in abin:
                rectangles.append((bin_count, rect.x, rect.y, rect.width, rect.height, rect.rid))
            bin_count += 1

        return rectangles

    def bin_list(self):
        """
        Return a list of the dimmensions of the bins in use, that is closed
        or open containing at least one rectangle
        """
        return [(b.width, b.height) for b in self]

    def validate_packing(self):
        for b in self:
            b.validate_packing()

    def reset(self): 
        # Bins fully packed and closed.
        self._closed_bins = collections.deque()

        # Bins ready to pack rectangles
        self._open_bins = collections.deque()

        # User provided bins not in current use
        self._empty_bins = collections.OrderedDict() # O(1) deletion of arbitrary elem
        self._bin_count = itertools.count()

In [154]:
class ShakerPackerSortedBBF(PackerOnline):
    """
    Rectangles aren't packed untils pack() is called
    """
    first_item = operator.itemgetter(0)

    def __init__(self, pack_algo=MaxRectsBssf, sort_key=lambda r: r[0]*r[1],rotation=False):
        """
        """
        super().__init__(pack_algo=pack_algo, rotation=rotation)
        
        self._sort_algo = lambda rectlist: sorted(rectlist, reverse=True, key=sort_key)

        # User provided bins and Rectangles
        self._avail_bins = collections.deque()
        self._avail_rect = collections.deque()

        # Aux vars used during packing
        self._sorted_rect = []

    def add_bin(self, width, height, count=1, **kwargs):
        self._avail_bins.append((width, height, count, kwargs))

    def add_rect(self, width, height, rid=None):
        self._avail_rect.append((width, height, rid))

    def _is_everything_ready(self):
        return self._avail_rect and self._avail_bins
    
    def add_rect_into_bin(self, width, height, rid=None):
 
        # Try packing into open bins
        fit = ((b.fitness(width, height),  b) for b in self._open_bins)
        #print(list(fit))
        fit = (b for b in fit if b[0] is not None)
        try:
            _, best_bin = min(fit, key=self.first_item)
            best_bin.add_rect(width, height, rid)
            return True
        except ValueError:
            return False    


    def pack(self):

        self.reset()

        if not self._is_everything_ready():
            print('Alarm')
            # maybe we should throw an error here?
            return

        # Add available bins to packer
        for b in self._avail_bins:
            width, height, count, extra_kwargs = b
            super().add_bin(width, height, count, **extra_kwargs)

        # If enabled sort rectangles
        self._sorted_rect = self._sort_algo(self._avail_rect)

        # Start packing
        
        # open new bins here!!!!!!!!!!!!!!!!!!!!!!!
        while len(self._sorted_rect) > 0:
            r = self._sorted_rect.pop(0)
            append_success = self.add_rect_into_bin(*r)
            if not append_success:
                new_bin = self._new_open_bin(*r)
                new_bin.add_rect(*r)

In [155]:
import Base.bp2DState
from Base.bp2DSimpleHeuristics import single_type_heuristic, first_fit, next_fit, most_enclosed_position, max_rest, \
    best_fit, first_fit_decreasing, next_fit_decreasing, get_all_heuristics, random_fit
from Base.bp2DState import State
from Base.bpReadWrite import ReadWrite
from Base.bp2DBox import Box
from Base.bpStateGenerators import random_state_generator, state_generator
import time

from Base.bp2DPlot import plot_packing_state
from Base.bp2DSimpleHeuristicsFunctions import most_enclosed_position_in_bin
from Base.bp2DState import Bin
from Base.bp2DState import Action, State
from Base.bpUtil import *
from Base.bpStateGenerators import random_state_generator
import matplotlib.pyplot as plt
from rectpack import SORT_AREA, PackingMode, PackingBin, newPacker

from rectpack import (MaxRectsBl, MaxRectsBssf, MaxRectsBaf, MaxRectsBlsf,
SkylineBl, SkylineBlWm, SkylineMwf, SkylineMwfl, SkylineMwfWm, SkylineMwflWm,
GuillotineBssfSas, GuillotineBssfSlas, GuillotineBssfLlas, GuillotineBssfMaxas,
GuillotineBssfMinas, GuillotineBlsfSas, GuillotineBlsfLas, GuillotineBlsfSlas, GuillotineBlsfLlas, 
                     GuillotineBlsfMaxas, GuillotineBlsfMinas, GuillotineBafSas, GuillotineBafLas,
                     GuillotineBafSlas, GuillotineBafLlas, GuillotineBafMaxas, GuillotineBafMinas)

In [156]:
count = 10
test_state = random_state_generator((10, 10),count,1,3,1,3)
path = f"test_instances/small_{count}"
    
ReadWrite.write_state(path=path, state=test_state)

In [157]:
packer = ShakerPackerSortedBBF(pack_algo=GuillotineBafMinas, rotation = False)
for box in test_state.boxes_open:
    packer.add_rect(box.w, box.h)
# Add the bins where the rectangles will be placed


In [158]:
packer.add_bin(*test_state.bin_size, float("inf"))
packer.pack()

In [90]:
new_bin = packer._new_open_bin(3, 4)

In [ ]:
packer.

In [91]:
new_bin.add_rect(3, 4)

R(0, 0, 3, 4)

In [159]:
packer.bin_list()

[(10, 10)]

In [93]:
new_bin.fitness(2, 2)

14

In [92]:
new_bin.rect_list()

[(0, 0, 3, 4, None)]

In [105]:
new_bin._sections

[R(0, 4, 3, 6), R(3, 0, 7, 10)]

In [125]:
bin = packer._open_bins[9]
w = 2
h = 2

In [128]:
bin._sections

[]

In [126]:
fit = ((bin._section_fitness(s, w, h), s, False) for s in bin._sections 
                if bin._section_fitness(s, w, h) is not None)

In [127]:
_, sec, rot = min(fit, key=operator.itemgetter(0))

ValueError: min() arg is an empty sequence

In [124]:
sec, rot

(R(0, 4, 3, 6), False)

In [113]:
list(fitn)

[(14, R(0, 4, 3, 6), False), (66, R(3, 0, 7, 10), False)]

In [106]:
packer._open_bins[9]._select_fittest_section(2,2)

(None, None)

In [58]:
len(packer)

10

In [ ]:
def ffd_shaker(state: State, plot_result=False, plot_steps=False, plot_name=None, speed_mode = False):
    '''ffd shaker'''
    start = time.time()
    step = 0
    name = 'ffd_shaker_plot'
    if plot_name is not None:
        name = plot_name
    sort_boxes_in_state(state)
    fill_state = False
    last_bin = 0
    while state.has_open_boxes():
        box = state.get_next_open_box()
        
        if fill_state:
        # in this state we try to fit smallest containers in remaining space, without creating new bins
            placement_success = False
            for i in range(last_bin, len(state.bins)):
                placement_success = state.place_box_in_bin(box, i)
                if placement_success:
                    break

            if not placement_success:
                state.boxes_open = state.boxes_open[::-1]
                fill_state = False
                if speed_mode:
                    last_bin = len(state.bins) - 1
                    
        else:
         # in this state we create containers, until find the container, which fits into created bins 
            placement_success = False
            for i in range(last_bin, len(state.bins)):
                placement_success = state.check_if_fits_somewhere_in_box(box, i)
                if placement_success:
                    state.boxes_open = state.boxes_open[::-1]
                    fill_state = True
                    break

            if not placement_success:
                state.open_new_bin()
                state.place_box_in_bin_at_pnt(box, -1, Point(0, 0))
            

        if plot_steps:
            plot_packing_state(state=state, step=step, fname=f"./vis/{name}_step")
            step += 1
    if plot_result:
        plot_packing_state(state, fname=f"./vis/{name}_result")
    state.solution_runtime = time.time() - start